# Import and set-up const.

In [45]:
import os
import sys
import dateparser
from bs4 import BeautifulSoup as bs
import requests
import re
import json
from tqdm import tqdm

import pandas as pd
import numpy as np

In [18]:
BASE_URL = r'https://www.myhora.com/ปฏิทิน/วันพระ-พ.ศ.{}.aspx'
YEAR_LIST = [2565, 2566, 2567]

TARGET_DIR = './data'

In [27]:
thai2eng_mapping = {}

for n in range(10):
    thai2eng_mapping[chr(ord('๐')+n)] = str(n)

# Extract info.

In [101]:
df = pd.DataFrame(columns=['date', 'weekday', 'day', 'month', 'year', 'kheun', 'ram', 'lunar_month', 'lunar_year', 'event'])

for year in YEAR_LIST:
    current_url = BASE_URL.format(year)
    response = requests.get(current_url)
    soup = bs(response.text, 'html.parser')

    bud_list = soup.find_all('div', class_='bud-day')

    looper = tqdm(bud_list,
                  desc=f"extracting from {year}",
                  unit="day")
    for day in looper:
        normal_date, lunar_date, event = bs.find_all(day, 'div')

        normal_date = dateparser.parse(normal_date.text)
        normal_date = normal_date.replace(year=normal_date.year-543)
        lunar_date = ''.join([c if c not in thai2eng_mapping else thai2eng_mapping[c] for c in lunar_date.text])
        event = event.text

        row = {
            'date' : normal_date,
            'weekday' : normal_date.weekday(),
            'day' : normal_date.day,
            'month' : normal_date.month,
            'year' : normal_date.year,
            'kheun' : -1 if 'ขึ้น' not in lunar_date else lunar_date.split(' ')[1],
            'ram' : -1 if 'แรม' not in lunar_date else lunar_date.split(' ')[1],
            'lunar_month' : -1 if 'เดือน' not in lunar_date else re.findall(r'.*\((\d+)\).*', lunar_date)[0],
            'lunar_year' : -1 if 'ปี' not in lunar_date else lunar_date.split('ปี')[1],
            'event' : event.strip('()') if event else None
        }
        df = pd.concat([df, pd.DataFrame([row])])

df.index = df['date']
df = df.drop(columns=['date'])

extracting from 2565:   0%|          | 0/51 [00:00<?, ?day/s]/var/folders/5g/160h3py942nb4pbbgy0dhtz40000gn/T/ipykernel_15787/4127778852.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])])
extracting from 2567: 100%|██████████| 51/51 [00:00<00:00, 539.36day/s]


In [102]:
df.head(10)

,weekday,day,month,year,kheun,ram,lunar_month,lunar_year,event
date,,,,,,,,,
2022-01-02,6,2,1,2022,-1,14,1,ฉลู,None
2022-01-10,0,10,1,2022,8,-1,2,ฉลู,None
2022-01-17,0,17,1,2022,15,-1,2,ฉลู,None
2022-01-25,1,25,1,2022,-1,8,2,ฉลู,None
2022-02-01,1,1,2,2022,-1,15,2,ฉลู,None
2022-02-09,2,9,2,2022,8,-1,3,ฉลู,None
2022-02-16,2,16,2,2022,15,-1,3,ฉลู,วันมาฆบูชา
2022-02-24,3,24,2,2022,-1,8,3,ฉลู,None
2022-03-02,2,2,3,2022,-1,14,3,ฉลู,None


In [105]:
if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)
df.to_csv(os.path.join(TARGET_DIR, 'wan-phra.csv'), index=True)